In [ ]:
# Structural Analysis of the disulfide bonds in 2q7q
# Author: Eric G. Suchanek, PhD.
# (c) 2023 Eric G. Suchanek, PhD., All Rights Reserved
# Last Modification: 2/16/23
# Cα N, Cα, Cβ, C', Sγ Å °

import pandas as pd
import numpy

import pyvista as pv
from pyvista import set_plot_theme

from Bio.PDB import *

import proteusPy
from proteusPy import *
from proteusPy.Disulfide import *
from proteusPy.DisulfideList import DisulfideList
from proteusPy.DisulfideLoader import Load_PDB_SS

# pyvista setup for notebooks
pv.set_jupyter_backend('trame')
#set_plot_theme('dark')


### Load the RCSB Disulfide Database
We load the database and get its properties as follows:

In [ ]:
PDB_SS = Load_PDB_SS(verbose=True)
PDB_SS.describe()

We see from the statistics above that disulfide 2q7q_75D_140D has the lowest energy, so let's extract it from the database and display it.

In [ ]:
best_ss = PDB_SS['2q7q_75D_140D']
best_ss.display(style='sb', light=False)

### Finding Potential Structural Relatives by Sequence Similarity
Working under the assumption that similar sequence -> similar function I generated a query on: https://www.ebi.ac.uk/pdbe/entry/pdb/2q7q to return PDB IDs for structures with high similarity. This yielded a ```.csv``` file, which we will import below:

In [ ]:
ss_df = pd.read_csv('2q7q_seqsim.csv')
ss_df.head(5)

All of the nearest sequence neighbors are sadly, bacterial. Let's extract the unique ids next.

In [ ]:
relative_list = ss_df['pdb_id'].unique()
relative_list

We now need to convert the list of PDB IDs into real disulfides from the database. We do this with the proteusPy DisulfideLoaderbuild_ss_from_idlist() function. Next we print out some relevant statistics.


In [22]:
relatives = DisulfideList([], 'relatives')

relatives = PDB_SS.build_ss_from_idlist(relative_list)


In [23]:
print(f'There are: {relatives.length} related structures.\nAverage Energy: {relatives.Average_Energy:.2f} kcal/mol\nAverage Ca distance: {relatives.Average_Distance:.2f} Å')
print(f'Average resolution: {relatives.Average_Resolution:.2f} Å \nAverage torsion distance: {relatives.Average_Torsion_Distance:.2f}°')

There are: 68 related structures.
Average Energy: 2.23 kcal/mol
Average Ca distance: 3.03 Å
Average resolution: 1.88 Å 
Average torsion distance: 123.83°


In [ ]:
ssmin, ssmax = relatives.minmax_energy()
duolist = DisulfideList([ssmin, ssmax], 'mM')
duolist.display(style='sb', light=False)

In [ ]:
duolist.display_overlay(light=False)

The two Disulfides

In [ ]:
close_neighbors = relatives.nearest_neighbors(ssmin, 20.0)


In [21]:
close_neighbors.length

27

In [ ]:
close_neighbors.display_overlay(light=False)

In [ ]:
ssTotList = PDB_SS.SSList
global_neighbors = ssTotList.nearest_neighbors(ssmin.chi1, ssmin.chi2, ssmin.chi3, ssmin.chi4, ssmin.chi5, 5.0)
global_neighbors.length

In [ ]:
global_neighbors.display_overlay(light=False)